# Workshop: Modeling and comparing brain cancer regulatory networks

In this workshop, you will learn how to model and analyze PANDA networks with the netZooR package. As a use case, we will compare primary brain cancers with recurrent brain cancer. The latter means a recurrence after removal of the primary tumor. Recurrent tumors are often more aggressive, as they arise from cells that resisted treatment. With network analysis, we aim to understand the regulatory mechanisms involved in this.

In **Module 1**, you will learn to model a regulatory network with PANDA. We will start with investigating the input motif prior, protein-protein interactions, and expression dataset and make sure to preprocess these datasets correctly so that they can be used with PANDA. Then, we will run PANDA on the complete dataset that includes both primary tumors and recurrences, to model an aggregate network representing all brain cancer samples as a whole. We will evaluate how changing various parameters in the PANDA program affect the output network.

**Module 2** will focus on comparative network analysis. First, we will run PANDA on primary and recurrent tumors separately, so that we obtain one aggregate network representing primary brain cancer and one aggregate network representing recurrent brain cancer. We will extract network edges that are most different between the two aggregate networks and look at the genes and transcription factors that are connected by these edges. We will also investigate some network centrality metrics (TF degrees and gene degrees) to check which TFs/genes differ most. After having compared the aggregate networks, we will apply LIONESS on the complete brain cancer dataset to estimate single-sample networks. We will calculate gene and TF degrees and compare these between the two sample groups of interest with a statistical approach (LIMMA).

Before we begin, we need to load the libraries we will use. This may take a few minutes.

In [ ]:
rm(list=ls())
suppressMessages(library("netZooR"))
suppressMessages(library("dplyr"))
suppressMessages(library("limma"))
suppressMessages(library("topGO"))

## Module 1: Modeling networks with PANDA

### *1.1 Examine the input datasets*

For this course, we prepared the .RData file input_data.RData that contains the datasets to be used as input in PANDA.

**Q1.** Load this file into R. What objects/variable are read in? Take a look at what is inside these objects, for example with the `head()` and `str()` functions. What do you think the different objects represent?

In [ ]:
load("input_data.RData")
ls()


PANDA's input consists of three data types. Below is some information on the datasets that we prepared for this workshop:

1. `prior` A "prior" regulatory network of putative TF-gene interactions. Here, we will be working with a motif prior that we previously built by scanning known TF motifs to promoter regions in the human genome. In this file, the first column indicates the TF, the second column the target gene, and the third column a "1" indicating a putative interaction.

2. `exp` Gene expression data. Gene expression data and gene annotations were downloaded from the Gene Expression Omnibus (GEO), accession ID GSE62153. This dataset contains gene expression data obtained from microarrays that were used to profile primary and recurrent brain cancer. These data were downloaded with the R package GEOquery and then normalized and log transformed. Rows indicate probe IDs and columns sample names.

3. `ppi` Potential protein-protein interactions (PPI) between TFs (optional). PPI data is optional in PANDA. Here, we downloaded data from StringDb, which contains scores in the range of [0,1000] indicating potential PPIs. We filtered these for TFs present in the motif prior and transformed scores to scores [0,1]. The first column indicates a TF, the second the TF it binds, and the third the interaction score.

Due to the message passing nature of PANDA, running the method on genome-wide data can be quite slow (depending on the available computing resources). We generally recommend to run PANDA on genome-wide data (i.e. including all genes for which expression data is available). However, to ensure it is feasible to run PANDA and explore its output multiple times during the limited time frame of this workshop, we subsetted the expression data to the so-called L1000 genes—978 "landmark" genes that should capture most of the variability in the human genome (see https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5990023/ for more information).

Finally, we also included the object `anno` that consists of annotation for the microarray probe identifiers and `clin` that includes clinical features for the patient samples used in this study. These were both obtained from the GEO dataset.

Let's examine the input datasets. We start with the motif prior.

**Q2.** Are there any duplicate rows in the motif prior? Hint: use the `duplicated()` function

In [ ]:
which(duplicated(prior))

**Q3.** How many unique TFs are there in the motif data?

In [ ]:
length(unique(prior[,1])) 

**Q4.** How many genes does each TF target on average?

In [ ]:
nrow(prior)/length(unique(prior[,1]))

**Q5.** *Bonus*. Make a plot of the degree distribution of the TFs (histogram of the number of genes targeted by each TF). Hint: look into the `table()` and `hist()` functions for a quick way to do this. We expect there to be a lot of TFs that target only a few genes and only a few TFs that target many genes. 

In [ ]:
hist(table(prior[,1]))

**Q6.** How many genes are in the motif data?

In [ ]:
length(unique(prior[,2]))

**Q7.** How many TFs target each gene on average?

In [ ]:
nrow(prior)/length(unique(prior[,2]))

**Q8.** Bonus. Look at the degree distribution of the genes. We expect most genes to only be targeted by a few TFs and a few of them to be targeted by many TFs. 

In [ ]:
hist(table(prior[,2]))

**Q9.** The density of a network indicates how many of all possible interactions are edges in the network. What is the original "density" of the motif prior network (calculate the percentage of prior interactions)?

In [ ]:
nrow(prior)/(length(unique(prior[,1]))*length(unique(prior[,2])))*100

Let's take a quick look at the PPI data.

**Q10.** How many unique TFs are present in the PPI data?

In [ ]:
length(unique(c(ppi[,1], ppi[,2]))) 

**Q11.** What is the average interaction scores for these PPIs?

In [ ]:
mean(ppi[,3])

**Q12.** Bonus. Plot a histogram of all PPI interaction scores.

In [ ]:
hist(table(ppi[,3]))

Finally, examine the expression data.

**Q13.** How many rows are in the expression data?

In [ ]:
nrow(exp)

**Q14.** For how many samples do we have expression data available?

*Note:* PANDA, like many other network reconstruction approaches, uses co-expression as one of its three input networks. Generally, the more input samples, the more accurate these co-expression measurements will be. While it is difficult to suggest a minimum sample size for running PANDA, as that depends on the biological question of interest, we generally recommend to include at least 10 samples and ideally >20 samples when using data for network modeling.

In [ ]:
ncol(exp)

### *1.2 Preprocessing of the PANDA input data*

In PANDA, row names from the gene expression dataset should match names of gene targets in the motif prior (second row in the motif prior). Here, we will convert the microarray probe IDs (ILMN_.....) to gene symbols, as these are used in the motif prior. We will use the annotation data provided in the anno object (see above).

**Q15.** How many unique probe IDs are in the annotation? How many unique gene symbols? How do you think this will affect converting probe IDs to gene symbols?

In [ ]:
length(unique(anno[,1])) 
length(unique(anno[,2]))

In R, row names must be unique. Therefore, we need to remove duplicated genes from our expression matrix. For this workshop, we will do this by, for each gene, choosing to keep the probe ID that has the highest variance across samples. If you finish early, feel free to revisit this decision and see how it changes the networks predicted by PANDA. To begin, we will calculate the standard deviation of the rows, and connect that to the original expression matrix:

In [ ]:
rowsd <- apply(exp, 1, sd)
exp <- cbind(rowsd, exp)

Then, we combine the gene expression data with its annotations. To do so, we will use the `merge()` function, matching the first column of the anno object with row names of exp.

In [ ]:
exp <- merge(anno, exp, by.x=1, by.y=0)

Let's sort the expression data based on the standard deviation of each probe, so that the most variable entries will be on the top.

In [ ]:
exp <- exp[order(exp$rowsd, decreasing=T),]

Now we can remove duplicate gene symbols with low variance. We can use the `str()` function to see what the `exp` object looks like now.

In [ ]:
idx <- which(duplicated(exp$Gene.symbol))
exp <- exp[-idx,]
str(exp)

**Q16.** Now substitute the row names of the exp object with gene symbols and then remove the probe ID, gene symbol, and rowsd columns so that the matrix only includes expression data. What are the dimensions of the preprocessed expression dataset?

*Note:* Like many other network reconstruction approaches, PANDA uses co-expression as one of its three input networks. The method therefore requires to have, for each gene, at least 3 samples with different values, so that Pearson correlation can be calculated. This is not a problem for the microarray example in this workshop—in fact, we just calculated the standard deviation for each gene and R did not return an error/NA (feel free to check what values are in the `rowsd` object). However, RNA-Seq data often includes genes with the same count across many samples (often, these are genes that are not detected in many samples and thus have counts of 0). Such genes are generally filtered out in the data preprocessing step before network modeling.


In [ ]:
row.names(exp) <- exp$Gene.symbol
exp <- exp[,-c(1:3)]
dim(exp)

Finally, before running PANDA, we subset our data to only include information common to both the motif prior and the gene expression data (same set of genes) and the motif prior and PPI (same set of TFs). While the PANDA algorithm can also run on data with missing values (e.g. gene targets that are present in the motif prior, but for which no expression data is available), it is generally a more robust approach to match the data prior to running PANDA. Note that subsetting can be done in the netZooR version of PANDA (check ?panda to see the remove.missing.data_type options). However, it is generally good practice to match the data prior to running PANDA, so that you exactly know what goes in and what comes out.

**Q17.** How many genes are in common between the motif prior and expression data?


In [ ]:
length(intersect(prior[,2], row.names(exp)))

To do the matching we first find the gene names that are common to both the motif prior and expression data and then filter both to only contain information for these genes.

In [ ]:
commonGenes <- intersect(prior[,2], row.names(exp))
prior <- prior[which(prior[,2] %in% commonGenes),]
exp <- exp[which(rownames(exp) %in% commonGenes),]

**Q18.** Now do the same for TFs in the motif prior and PPI data. How many TFs are in common?

In [ ]:
length(intersect(unique(prior[,1]), unique(c(ppi[,1],ppi[,2])))) # 568
commonTFs <- intersect(prior[,1], c(ppi[,1],ppi[,2]))
prior <- prior[which(prior[,1] %in% commonTFs),]
ppi <- ppi[which(ppi[,1] %in% commonTFs & ppi[,2] %in% commonTFs),]

**Q19.** What are the dimensions of the subsetted motif prior, PPI, and expression data? How many unique TFs and genes are present in each dataset?

In [ ]:
dim(prior)
dim(ppi) 
dim(exp) 
length(unique(prior[,1]))
length(unique(prior[,2]))
length(unique(ppi[,1])) 
length(unique(ppi[,2])) 
nrow(exp)

### *1.3 Running PANDA*

Now that we have all the data in the right format, running PANDA is fairly straightforward:

In [ ]:
net <- panda(prior, exp, ppi, progress=T, hamming=0.01)

Look at the output from `panda()` and answer the following questions:

**Q20.** How many message-passing steps did it take for PANDA to converge?

23

**Q21.** How long did it take to run? Note that running times will depend on the computational resources that are available.

PANDA estimates relationships in three types of networks: (1) regulatory (TF-gene interactions), (2) co-regulatory (gene-gene interactions), (3) co-operativity (TF-TF interactions). These can be accessed with the `slot()` function. In this workshop, we will focus on the regulatory network estimated by PANDA:

In [ ]:
RegNet <- slot(net, "regNet")

Let's visualize this network in a few different ways to get a sense of the information they contain:

**Q22.** Make a histogram of the edge weights in the regulatory network. Hint: use the `hist()` function.

In [ ]:
hist(RegNet,100)

**Q23.** Make a heatmap of the edge weights in the regulatory network. In this heatmap, the TFs will be columns and the genes will be rows.

In [ ]:
heatmap.2(RegNet, Rowv = FALSE, Colv = FALSE, trace = FALSE, density.info = "none")

Congratulations on finishing module 1! Feel free to take a short break and/or ask any remaining questions before continuing with module 2.

## Module 2: Comparative network analysis

### *2.1 Running PANDA on primary and recurrent tumors*

In the previous module, we modeled one aggregate regulatory network on the entire expression dataset. Here, we will focus on comparative network analysis between two aggregate networks. To do so, we need to start looking at the clinical data. Clinical features associated with our expression dataset can be found in the `clin` object.

**Q24.** Take a look at the `clin` object. Which column contains information on tumor type (primary/recurrent)?

In [ ]:
head(clin)

**Q25.** How many primary and how many recurrent tumors does the data contain?


In [ ]:
table(clin$type)

Use information from the clinical data to make two expression objects, one for primary and one for recurrent tumors. Then run PANDA on both datasets with default parameters.

In [ ]:
idx_prim <- which(clin$type=="Primary_GBM")
samples_prim <- row.names(clin[idx_prim,])
exp_prim <- exp[,which(colnames(exp) %in% samples_prim)]
net_prim=panda(prior, exp_prim, ppi, progress = T)
RegNet_prim=slot(net_prim, 'regNet')

idx_recur <- which(clin$type=="Recurrent_GBM")
samples_recur <- row.names(clin[idx_recur,])
exp_recur <- exp[,which(colnames(exp) %in% samples_recur)]
net_recur=panda(prior, exp_recur, ppi, progress = T)
RegNet_recur=slot(net_recur, 'regNet')

**Q26.** How many message-passing steps did it take for PANDA to converge on the primary tumors? And on the recurrences?

### *2.2 Comparing two PANDA networks*

Now we are ready to compare the two PANDA networks. The `panda()` function in netZooR outputs each network of a matrix, with TFs in rows and their target genes in columns. To easily visualize differential edges and the TFs and genes connected to them, we will transform this into an edgelist using the code below. We first combine the TF and gene names in the same order as the `c()` function in R unlists matrices, and convert that into a `data.frame`.

In [ ]:
tfnamerep <- rep(row.names(RegNet_prim), ncol(RegNet_prim))
genenamerep <- rep(colnames(RegNet_prim), each=nrow(RegNet_prim))
edgelist <- cbind(tfnamerep, genenamerep)
edgelist <- as.data.frame(edgelist)

Now, we are ready to attach the edges in the primary and recurrent networks to this edge list.

In [ ]:
edgelist <- cbind(edgelist, c(RegNet_prim), c(RegNet_recur))
colnames(edgelist) <- c("TF","Gene","Primary","Recurrent")

**Q27.** Visualize edge weights in primary vs recurrent tumors in a scatterplot. Hint: use `plot()`

In [ ]:
plot(edgelist[,3:4], pch=16)

It is often very useful to determine the edges that best define the differences between two networks. In this case, we would like to select edges that are both likely to be "real" in a particular network (i.e. have a high edge weight) as well as "different" between the two networks. Because the weights of PANDA edges can be thought of as z-score units, edge-weights can be easily converted into probability values. Under this framework, we can multiply the probability that an edge is "real" by the probability that it is "different" between two networks to get a combined probability score representing whether an edge is both real and different. We can then select edges based on this combined probability score. In our example, we use the PANDA output matrices to select edges that have at least a 75% chance of being both real and different:

In [ ]:
pprim <- pnorm(RegNet_prim)
precur <- pnorm(RegNet_recur)
prim_spec <- pprim*pnorm(RegNet_prim-RegNet_recur)>0.75
recur_spec <- precur*pnorm(RegNet_recur-RegNet_prim)>0.75

We can then port these to our edgelist as follows:

In [ ]:
idx_prim <- which(c(prim_spec))
idx_recur <- which(c(recur_spec))
prim_spec_edges <- edgelist[idx_prim,]
recur_spec_edges <- edgelist[idx_recur,]

**Q28.** Visualize these tumor type-specific edges on top of the scatterplot you made in Q6. Hint: use `point()` to overlay points of different colors on the scatterplot.

In [ ]:
plot(edgelist[,3:4], pch=16)
points(prim_spec_edges[,3:4], pch=16, col="blue")
points(recur_spec_edges[,3:4], pch=16, col="red")

Let's take a look at the top differential edges.

**Q29.** How many edges are specific to the primary network? And to the recurrent network?

In [ ]:
nrow(prim_spec_edges)
nrow(recur_spec_edges)

Calculating the TF out-degree and gene in-degree is easiest to do on the direct output from the `panda()` function in netZooR.

**Q30.** The degree is defined by the sum of all edge weights, either pointing from a TF (TF out-degree), or pointing to a gene (gene in-degree). Calculate TF and gene degrees in each network. What is the range of out-degree values in each network? And of the in-degree? Hint: you can use the `apply()` and `sum()` functions in R to calculate degrees (similar to what we used above to calculate the standard deviation across rows in a matrix).

move up to the first time we mention degree distribution

In [ ]:
outdegree_prim <- apply(RegNet_prim, 1, sum)
outdegree_recur <- apply(RegNet_recur, 1, sum)
indegree_prim <- apply(RegNet_prim, 2, sum)
indegree_recur <- apply(RegNet_recur, 2, sum)
min(outdegree_prim); min(outdegree_recur) # outdegree range ~[-300, 1900]
max(outdegree_prim); max(outdegree_recur)
min(indegree_prim); min(indegree_recur) # indegree range ~[-120, 400]
max(indegree_prim); max(indegree_recur)

**Q31.** Calculate degree difference and return the top 3 TFs and target genes with highest difference between primary and recurrent. Look up some of these genes and their functions in GeneCards (https://www.genecards.org/). Do you find any interesting TFs/genes that could play a role in brain cancer?

*Note:* Instead of looking up individual edges and genes, it is also possible to use gene set enrichment analysis to check if there are any differentially regulated pathways between your networks of interest. Differences in gene degrees often highlight important dysregulated pathways. We will do this to analyze the LIONESS results (see bonus section below). If you end the workshop early, feel free to see if you can run the enrichment analysis in the aggregate networks as well.

In [ ]:
out_diff <- outdegree_prim-outdegree_recur
names(out_diff[order(out_diff, decreasing=T)])[1:3] # "NKX2-5" "MIXL1"  "VAX1" (primary)
names(out_diff[order(out_diff, decreasing=F)])[1:3] # "DNMT1" "NR0B1" "KLF6" (recurrent)
in_diff <- indegree_prim-indegree_recur
names(in_diff[order(in_diff, decreasing=T)])[1:3] # "RPN1"  "EPHA3" "MMP2" (primary)
names(in_diff[order(in_diff, decreasing=F)])[1:3]

### *BONUS MODULE 1: Exploring PANDA's hyperparameters*

There are several parameters in the `panda()` function that are worth investigating. Let's try running PANDA for a few different settings for these parameters to see how it affects the output.

**Q32.** Run `panda()` as above except with `hamming=0.1`.
1. How many message passing iterations were used?
2. How long did the algorithm run?

In [ ]:
net=panda(prior, exp, ppi, progress = T, hamming=0.1)

**Q33.** Run `panda()` as above in section 1.3, except with `alpha=0.3`.
1. How many message passing iterations were used?
2. How long did the algorithm run?
3. Make a histogram of the regulatory network weights predicted with `alpha=0.3`. Edge weights from PANDA often have a "bimodal" distribution, meaning that two peaks can be observed in the distribution. Generally, most edges in the motif prior obtain high weights after message passing, while edges that are not in the motif prior obtain low weights. How does the "overlap" of the two modes of the distribution change in this PANDA run compared to the original PANDA run with the default `alpha=0.1`?
4. Spend a bit of time playing with the alpha parameter and investigate how it affects the predicted regulatory network edge weights.

In [ ]:
net=panda(prior, exp, ppi, progress = T, hamming=0.01, alpha=0.3)

In [ ]:
RegNet=slot(net, 'regNet')
hist(RegNet,100)

In [ ]:
for(i in c(0.05, 0.1, 0.2, 0.4, 0.8)){
	net=panda(prior, exp, ppi, progress = T, hamming=0.1, alpha=i)
	RegNet=slot(net, 'regNet')
	hist(RegNet,100, main=paste("alpha=", i, sep=""))
	print(i)
}

Some more explanation

### *BONUS MODULE 2: Modeling patient-specific networks with LIONESS*

While comparing two aggregate networks can point to differential regulation between primary tumors and recurrences, modeling and analyzing patient-specific networks is more powerful as it captures patient heterogeneity. Let's therefore model networks for all patients in this dataset. This can be done as follows:

In [ ]:
lionessRes <- lioness(exp, prior, ppi, progress=F, hamming=0.05)

This should run in about 5-10 minutes, so feel free to take a break until the networks are ready.

*Note:* We set `progress=F` to not output information on the message passing iterations this time and keep the printed output on the screen clean. In addition, for this tutorial we set `hamming=0.05` to stop network convergence a bit early, as with default settings it would take too much time to model all networks for the workshop. However, in future applications of PANDA+LIONESS to your own data, default options are recommended.

Let's have a look at the LIONESS output. This is an object of the "list" class.

**Q34.** How long is this list?

In [ ]:
length(lionessRes)

**Q35.** Check the first element in the list. Hint: this can be access using `lionessRes[[1]]`. What are the dimensions of this element and what does the data look like? What do you think this element represents?

In [ ]:
dim(lionessRes[[1]]) # 568 901
lionessRes[[1]][1:4,1:5]

**Q36.** Calculate the gene degree for the first sample and visualize this in a histogram. How are these degrees distributed?

In [ ]:
indegree_sample1 <- apply(lionessRes[[1]], 2, sum)
hist(indegree_sample1, breaks=100)

### *2.4 Comparative network analysis with LIONESS*

We will perform a comparative analysis on gene degrees with LIMMA. Let's start by calculating the indegrees with the code below. First, we will define a new function that will calculate the indegree by applying the sum function across columns:

In [ ]:
indegreefun <- function(x, ...){
	apply(x, 2, sum)
}

Next, we can apply this function to the `lionessRes` list to obtain the degrees for each sample, in list format. We do this with the `lapply()` function, which basically applies a function (our `indegreefun()`) to a list:

In [ ]:
indegrees <- lapply(lionessRes, indegreefun)

Finally, we transform this list into a matrix and add back in the row and column names.

In [ ]:
indegrees <- matrix(unlist(indegrees),ncol=length(indegrees))
row.names(indegrees) <- colnames(lionessRes[[1]])
colnames(indegrees) <- colnames(exp)

**Q37.** Take a look at the dimensions of this matrix and its column and row names. Does it look similar to any of the previous objects you worked with?

In [ ]:
dim(indegrees)
indegrees[1:4,1:5]

Now that the input data is ready, let's prepare the input material for LIMMA. LIMMA is very popular for differential expression analysis. As our gene degree data is similar to expression data, we can apply the method on our `indegrees` object. First, we need to design the comparison model that tells the program which groups to compare. We will "contrast" the group of recurrences with the group of primary tumors:

In [ ]:
targets <- cbind(row.names(clin), clin$type)
group <- factor(targets[,2])
design <- model.matrix(~0+group)
cont.matrix <- makeContrasts(recurvsprim = (groupRecurrent_GBM - groupPrimary_GBM), levels = design)

**Q38.** Take a look at the `design` and `cont.matrix` objects. What do you think is represented in these objects?

*Note:* With LIMMA, various more complex designs are possible, for example to correct for covariates (e.g. features that may confound the results such as the patient's age or sample batch), run comparisons between more than two groups, or time series analysis. This goes beyond the scope of this workshop, but may be good to know about in case you want to analyze a more complex experiment.

We can now fit the LIMMA model as follows:

In [ ]:
fit = lmFit(indegrees, design)
fit2 = contrasts.fit(fit, cont.matrix)
fit2e = eBayes(fit2)
toptable<-topTable(fit2e, number=nrow(indegrees), adjust="fdr") 
colnames(toptable)[1:2] <- c("Degree_diff", "Avg_degree")

**Q39.** LIMMA sorts the output on the most significant differences. The `adj.P.Val` column contains p-values after correcting for multiple testing. Did we detect any significant differentially regulated genes in this comparison?

Even when we might not find significantly differentially regulated genes, we can still look at the top genes to see if there they collectively are enriched for differential regulation between the two sample groups. This can be done with approaches such as Gene Ontology enrichment analysis with the `topgo()` package.

First, load the GO terms for Homo Sapiens in R:

In [ ]:
GO2Symbol.bp <- annFUN.org("BP", mapping = "org.Hs.eg.db", ID = "symbol")
symbol2GO.bp <- inverseList(GO2Symbol.bp)

Now load the following function in R:

In [ ]:
runtopGO <- function(x=background, y=interesting, ...){
	geneList <- factor(as.integer(x %in% y))
	names(geneList) = toupper(x)
	GOdata.bp <- new("topGOdata", ontology = "BP", allGenes = geneList, 	geneSel = geneList, annot = annFUN.gene2GO, gene2GO = symbol2GO.bp)
	resultFisher = runTest(GOdata.bp, algorithm="classic", 	statistic="fisher")  
	enrichRes.bp <- GenTable(GOdata.bp, classic = resultFisher, orderBy = 	"classic", ranksOf = "fisher", topNodes = length(score(resultFisher)))
	enrichRes.bp <- enrichRes.bp[which(enrichRes.bp$Annotated>10 & 	enrichRes.bp$Annotated<=50 & enrichRes.bp$Significant>=3),]
	adjP <- p.adjust(enrichRes.bp$classic,"BH")
	enrichRes.bp <- cbind(enrichRes.bp, adjP)
	enrichRes.bp <- enrichRes.bp[order(enrichRes.bp$adjP),]
	enrichRes.bp[which(enrichRes.bp$adjP<0.15),]
}

This function checks for enrichment of GO terms in a list of with the topGO package. The input is a vector of "interesting" genes compared to a "background" gene list. We will define the latter based on the genes we ran our network models on:

In [ ]:
background <- row.names(indegrees)

*Note:* The function above also includes some thresholds that we defined for the purpose of this tutorial. Basically, we filter all GO terms based on gene set size (we only look into gene sets of 10-50 genes, as often smaller/larger gene sets are very specific/general and less informative). We only select gene sets that include at least 3 genes from our "interesting" list. Finally, we use an FDR (adjP) threshold of 0.15. While the most widely used p-value threshold is probably 0.05, FDRs can generally be a bit higher. Basically, an FDR of 15% means that you expect 85% of your genes to be true positives. Such thresholds are again dependent on the research question of interest and how/whether you plan to validate your results.

Of course, if you are familiar with the code, feel free to update this function, adding parameters that allow you to change these thresholds, and then test various thresholds to see how it would affect your results.

**Q40.** Take the top 25 genes that have higher targeting in the recurrent samples and input this as run topGO with the code above. What terms are significant with FDR<0.15?

In [ ]:
topgenes_recur <- row.names(toptable[which(toptable$Degree_diff>0),])[1:25]
runtopGO(x=background, y=topgenes_recur)

**Q41.** Do the same for genes that have higher targeting in the primary samples. What GO terms are significant? Do you think these are relevant in the context of cancer? If the "Term" is not complete, feel free to search for the GO.ID online to get more information on the specific GO term.

In [ ]:
topgenes_prim <- row.names(toptable[which(toptable$Degree_diff<0),])[1:25]
runtopGO(x=background, y=topgenes_prim)

Congratulations, you have made it to the end of this module! If you find you have time to spare, feel free to further explore the LIONESS networks, for example by comparing TF out-degrees or edge weights.